### Installing openEO client libraries

To use openEO, you need to make sure that the openEO client libraries are installed. The library is available on pypi, so it can be installed with pip: 
https://pypi.org/project/openeo/

We recommend using at least Python 3.6 so in the notebook environment, the install command is:

In [1]:
!pip3.6 install --user openeo

Looking in indexes: https://artifactory.vgt.vito.be/api/pypi/python-packages/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
## loading of the required functions 

import openeo
import json
import os
import geopandas as gpd
import pandas as pd

### Create openeo session

In [3]:
endpoint = os.environ.get("ENDPOINT", "https://openeo-dev.vito.be")
openeo_url = "{e}/openeo/{v}".format(e=endpoint, v="1.0.0") 
openeo_session = openeo.connect(openeo_url).authenticate_oidc(provider_id="egi") # The first time you will try to connect to openeo you will need to link your Terrascope account to EGI credentials. Follow the procedure list below to allow this. 
#if you want more info, please consult: https://docs.terrascope.be/#/Developers/WebServices/OpenEO/OpenEO?id=logging-in 

Authenticated using refresh token.


### Description on about the harvest detection service

### Crop calendar (harvest prediction)
The CropCalendar service is used to automatically predict harvest dates from a set of input geometries. The harvest date prediction is written in an additional attribute table of the input geometry collection. The output of the service is a geojson format of all the input geometries including the harvest date as an addtional column.

### Methodology
A trained neural network model is used to automatically predict harvest dates based on input timeseries of Sentinel data. Both timeseries generated from **Sentinel-1 and Sentinel-2** are used to ingest in the model, more specifically the **VH/VV** ratio and the **daily fAPAR** (from CropSAR) are used. The model is only trained on detecting the moment the **above ground biomass** is removed from the field. 

### Quality
The model was trained on field data from Belgium with a focus on potato, maize and flax. Overall, the harvest date could be detected within **5-6 days accurate** based on validation of some reference fields in Italy (sugarbeet and soybean). However, it can be expected that for certain crop types and areas with different field management around harvest the performance might be less accurate. Furthermore, if the harvest of a certain crop does not match with the removal of all above ground biomass, the harvest detector will detect the acutal removal of all biomass. This is for example the case for cotton.

### Disclaimer
The time range parameter should span more than 2 months and preferably centered around the expected harvest date otherwise the performance will drop. Requesting harvest date prediction in near-real time modus is not yet supported. The input geometries should be larger than 20x20 m since a 10m inwards buffer is automatically taken to minimize boundary contamination in the timeseries.

### Links
For a more detailed description of the algorithm, please consult:

[Link]https://blog.vito.be/remotesensing/what-happens-on-the-fields-monitoring-the-crop-calendars"

### Required parameters

* **date**: The period for which satellite data should be retrieved and used to estimate harvest date(s)
  * e.g. "2017-01-01","2019-12-31".  
* **polygon**: Featurecollection of polygons for which harvest should be estimated

### Output
The output of the service is a GEOJSON object with all the harvest date(s) that are detected for each field. 
The labels of the fields are used to assign the harvest date(s) to. In case no harvest event could be detected, it is set to 'None'.

In [4]:
# Define input parameters
 
time_range = "2020-01-31", "2020-12-31"

# now we only need to still specify the polygons for which the predictions should be made
# please first upload the geometry files to the Private/Public folder in the notebook environment!

ID_identifier = 'id' # the column name of the input geometries files that contains the unique names of the fields. 
base_public_folder = os.path.join('/data', 'users', 'Public', os.environ['USER']) # use the public folder to store the input geometries so that they can be loaded from there
output_folder = 'Harvest_output_fields_demo' # name of the subfolder in the working directory that will be used to store the results
os.makedirs(os.path.join(os.getcwd(), output_folder), exist_ok = True)


file_geom_path = r'Public/e_shape/Notebooks/test_fields/test_fields_WIG2020_3_fields.shp' #Copy the file path from the available shapefile(s) on the left side of the screen. Do this by right clicking on the file and click on 'Copy Path'
file_geom_path = file_geom_path.replace('Public/','') # remove redundant Public name in directory
full_geom_path = os.path.join(base_public_folder, file_geom_path)

# now that we have the path, the shapefile can be loaded and coverted to the desired format to send to openEO
gpd_shp = gpd.read_file(full_geom_path)
field_ids = gpd_shp[ID_identifier].to_list()
polygons = json.loads(gpd_shp.to_json()) # Mandatory parameter

Harvest_process = openeo_session.datacube_from_process("CropCalendar", namespace="https://openeo-dev.vito.be/openeo/1.0/processes/u:bontek/CropCalendar"
                                                         , date=time_range ,polygon = polygons) #create the process graph for the service


#print(Harvest_process.graph)


# Obtain result

### option 1 -> When executing for many fields over a long time period (do asynchronous call)
Harvest_result = Harvest_process.send_job().start_and_wait().get_result().load_json() # # once the job is launched it will take a while before you will get the result so wait until in the screen  below the message 'finished' pops-up. First you will see 'queued' -> 'running' -> 'finished'
    # in case the message is 'error' -> Please try again to run the job once again, if still fails please reach out.  

### option 2 -> When executing for only few fields and for a short time period, a synchronous call can be done which is quite fast

#Harvest_result = Harvest_process.execute()

/home/bontek/.local/lib/python3.6/site-packages/openeo/metadata.py:240: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


0:00:00 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': send 'start'
0:01:09 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:01:14 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:01:21 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:01:29 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:01:39 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:01:52 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:02:08 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': queued (progress N/A)
0:02:27 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': running (progress N/A)
0:02:51 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': running (progress N/A)
0:03:22 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': running (progress N/A)
0:04:00 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': running (progress N/A)
0:04:47 Job '9b820d0c-68e3-4fd5-9180-ca5d47125c59': running (progress N/A)
0:05:46 Job '9b820d0c-68e3-4fd5-9180-ca5d4

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated method get_result. (Use `RESTJob.get_results` instead.) -- Deprecated since version 0.4.10.


### Store the output result

In [11]:
Harvest_dates = []
for field in Harvest_result['features']:
    harvest_field = field['properties']['Harvest_date']
    Harvest_dates.append(harvest_field)
    
df_harv_result = pd.DataFrame(data = Harvest_dates, index = field_ids, columns = ['Harvest_date'])     
df_harv_result.to_csv('Harv_result.csv', index = True) # This stores the CSV file in the same folder as the notebook is stored
print(df_harv_result)

                     Harvest_date
AW_wZ9XOQA72oZUZfVrl         None
AW_waRfcNgS9FFefG0bM         None
AW_waDXzQA72oZUZfVrr         None
